In [2]:
import pandas as pd
from datasets import Dataset
import pickle
from utils import id2label, label2id, classes

In [4]:
df = pd.read_json('all.json')
history_dataset = df[df.genre=='history']
titles = history_dataset['title'].to_list()
DATA = Dataset.from_pandas(history_dataset)

In [6]:
def predict_logreg(model_path='models/logreg.pkl', vect_path='models/tfidf_vectorizer.pkl'):

    # load
    with open(model_path, 'rb') as f:
        model = pickle.load(f)

    with open(vect_path, 'rb') as f:
        vect = pickle.load(f)

    x_test = vect.transform(DATA['text'])
    pred = model.predict_proba(x_test)
    
    return pred

In [7]:
pred_logreg = predict_logreg('models/logreg_stopwords.pkl', 'models/tfidf_vectorizer_stopwords.pkl')

In [8]:
total = 0
new_history = []
i = 0
for sample in pred_logreg:
    if sample[4] > 0.9:
        total += 1
        new_history.append(DATA[i])
    i += 1
        
total

348

In [9]:
DATA[52]

{'title': 'I',
 'genre': 'history',
 'text': "This able prince perished with his son, Osfrid, in a great battle which he fought against Penda, king of Mercia, and Caedwalla, king of the Britons. That event, which happened in the forty-eighth year of Edwin's age and seventeenth of his reign, divided the monarchy of Northumberland, which that prince had united in his person. Eanfrid, the son of Adelfrid, returned with his brothers, Oswald and Oswy, from Scotland, and took possession of Bernicia, his paternal kingdom: Osric, Edwin's cousin-german, established himself in Deïri, the inheritance of his family; but to which the sons of Edwin had a preferable title. Eanfrid, the elder surviving son, fled to Penda, by whom he was treacherously slain. The younger son, Vuscfraea, with Yffi, the grandson of Edwin, by Osfrid, sought protection in Kent, and not finding themselves in safety there, retired into France to King Dagobert, where they died.",
 'labels': 4,
 '__index_level_0__': 2186}

In [10]:
new_history_df = df.from_dict(new_history).rename(columns={'__index_level_0__':'index'}).set_index('index')

To make sure that there are no paragraphs in the resulting data that the classifyer was trained on:

In [21]:
# old dataset
random_dataset = df[df.genre=='history'].sample(n=500, random_state=13)

In [26]:
# everything that is only in the new dataset
good_indexes = list(set(new_history_df.index) - set(random_dataset.index))

In [33]:
new_history_df = new_history_df.loc[good_indexes]

In [34]:
new_history_df

,title,genre,text,labels
index,,,,
6147,LX,history,"The dominion, which England claimed over Irela...",4
6150,LX,history,"Meanwhile, the unfortunate king was necessitat...",4
4105,XXXIII,history,"HENRY, BEING DETERMINED to avenge himself on t...",4
4106,XXXIII,history,"The duke of Norfolk, meanwhile, began to move ...",4
4110,XXXIII,history,"James earl of Arran, of the name of Hamilton, ...",4
...,...,...,...,...
4025,XXXI,history,"The trial and conviction of queen Anne, and th...",4
4038,XXXI,history,The duke of Norfolk was appointed general of t...,4
4039,XXXI,history,"Soon after this prosperous success, an event h...",4


In [35]:
new_history_df.to_json('hume_json/history_automatic.json')